****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [2]:
from xml.etree import ElementTree as ET
import pandas as pd

In [3]:
doc = ET.parse( './data/mondial_database.xml' )
root = doc.getroot()

In [4]:
root

<Element 'mondial' at 0x000001D62A682548>

# 1) Find 10 countries with the lowest infant mortality rates

In [5]:
root.find('country')

<Element 'country' at 0x000001D62A682598>

In [6]:
root.find('country//infant_mortality')

<Element 'infant_mortality' at 0x000001D62A682958>

In [7]:
# create empty list
infant_mortality_rate = []
# get infant mortality rate for each country: root element - mondial, element - country, subelement - infant mortality,
# and add it to the list
for country in root.iterfind('country[infant_mortality]'):
    name = country.find('name').text
    rate = country.find('infant_mortality')   
    infant_mortality_rate.append([name, float(rate.text)]) 

In [8]:
# convert the list to data frame and assign column names
df = pd.DataFrame(infant_mortality_rate, columns=['country', 'rate'])
df.sort_values(by='rate').head(10)

,country,rate
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


# 2) Find 10 cities with the largest population

In [9]:
root.find('country//city')

<Element 'city' at 0x000001D62A693048>

In [10]:
root.find('country//city//population')

<Element 'population' at 0x000001D62A6931D8>

In [11]:
root.find('country//city//population[@year="2011"]')

<Element 'population' at 0x000001D62A693278>

In [12]:
# create empty dictionary
dict_cities = {}
# get country and population in 2011 for each city: element - country, subelement - city, subelement(city) - population,
for cities in doc.findall('country/province/city'):
    max_year = 0
    city_population = 0
    for city in cities:
        if city.tag == 'name':
            value = city.text
        elif city.tag == 'population':
            # find city element with attribute year and choose latest year 
            if int(city.attrib['year']) > max_year:
                max_year = int(city.attrib['year'])
                city_population = int(city.text)
    dict_cities[city_population] = value

In [13]:
# convert the dictionary to data frame and assign column names
df = pd.DataFrame.from_dict(dict_cities, orient='index').reset_index()
df.rename(columns={'index': 'population', 0: 'city'}, inplace=True)
df_columns = df.sort_values(by='population', ascending=False).head(10)
df_columns[['city', 'population']]

,city,population
298,Shanghai,22315474
2012,Istanbul,13710512
1831,Mumbai,12442373
366,Moscow,11979529
52,Beijing,11716620
1184,São Paulo,11152344
1592,Tianjin,11090314
3,Guangzhou,11071424
2604,Delhi,11034555
2377,Shenzhen,10358381


# 3) Find 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [14]:
# ethnic groups with the largest population
dict_ethnic_group = {}
for countries in doc.findall('country'):
    max_year = 0
    population = 0
    for country_item in countries:
        if country_item.tag == 'population':
            if int(country_item.attrib['year']) > max_year:
                max_year = int(country_item.attrib['year'])
                population = int(country_item.text)
        if country_item.tag == 'ethnicgroup':
            name = country_item.text
            percent = float(country_item.attrib['percentage']) / 100
            try:
                dict_ethnic_group[name] += int(population * percent)
            except(KeyError):
                dict_ethnic_group[name] = int(population * percent)


In [15]:
df = pd.DataFrame.from_dict(dict_ethnic_group, orient='index').reset_index()
df.rename(columns={'index': 'ethnic group', 0: 'population'}, inplace=True)
df_columns = df.sort_values(by='population', ascending=False).head(10)
df_columns

,ethnic group,population
42,Han Chinese,1245058800
266,Indo-Aryan,871815583
110,European,494872201
86,African,318325104
265,Dravidian,302713744
106,Mestizo,157734349
270,Bengali,146776916
18,Russian,131856989
238,Japanese,126534212
219,Malay,121993548


# 4) Find name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [16]:
# river, lake, airport
def country_decode(codes):
    # Takes a string of country codes as an argument and returns a list of long-form country names.
    countries = []
    codes = codes.split()
    for code in codes:
        countries.append(concodes[code])
    return countries

def find_max(feature, metric):
    # Takes two strings as arguments, returns a tuple of the name of the feature, the max of the metric, and the 
    # countries associated with it.
    max_m = 0
    codes = ''
    max_name = ''
    for f in doc.findall(feature):
        for node in list(f):
            if node.tag == 'name':
                name = node.text
            if node.tag == metric:
                try:
                    met = float(node.text)
                except TypeError:
                    met = 0
                if met > max_m:
                    max_m = met
                    codes = f.attrib['country']
                    max_name = name
        countries = country_decode(codes)
    return max_name, max_m, countries
        
concodes = {}
for c in doc.findall('country'):
    code = c.attrib['car_code']
    for node in list(c):
        if node.tag == 'name':
            name = node.text
        concodes[code] = name
        
riv = find_max('river', 'length')
print('The longest river is ' + riv[0] + ', which is ' + str(int(riv[1])) + 'km long.')
print('It runs through: ')
for e in riv[2]:
    print(e)

lake = find_max('lake', 'area')
print('\nThe largest lake is ' + lake[0] + ', with an area of ' + str(int(lake[1])) + 'km^2.')
print('It borders: ')
for e in lake[2]:
    print (e)

air = find_max('airport', 'elevation')
print('\n' + air[0] + ' in ' + str(air[2][0]) + ' is the highest airport, at an elevation of ' + str(air[1]) + ' meters.')

The longest river is Amazonas, which is 6448km long.
It runs through: 
Colombia
Brazil
Peru

The largest lake is Caspian Sea, with an area of 386400km^2.
It borders: 
Russia
Azerbaijan
Kazakhstan
Iran
Turkmenistan

El Alto Intl in Bolivia is the highest airport, at an elevation of 4063.0 meters.
